In [1]:
from pyspark import SparkConf, SparkContext
from collections import OrderedDict
from functools import partial
import numpy as np

In [2]:
partitions = 24
parcsv = sc.textFile("/home/milroy/pyspark/2014-04-24.csv", partitions)
parcsv.take(5)

[u'oss07;lustre.scratch.ost.obdfilter.OST0017.cache_access;0.00000;float;pages/s;1398382546',
 u'oss07;lustre.scratch.ost.obdfilter.OST0015.disconnect;0.00000;float;requests/s;1398382546',
 u'oss07;cpu_intr;0.0;float;%;1398382546',
 u'oss07;lustre.scratch.ost.obdfilter.hosttotal.cache_access;0.00000;float;pages/s;1398382546',
 u'oss07;lustre.scratch.ost.obdfilter.OST0025.connect;0.00000;float;requests/s;1398382546']

In [3]:
filtered = parcsv.filter(lambda line: len(line.split(';')) == 6)

In [4]:
def cast(line):
    try:
      val = float(str(line.split(';')[2]))
    except:
      val = 0.0
    return (int(line.split(';')[5]), line.split(';')[0], 
            line.split(';')[1], val)

In [5]:
parsed = filtered.map(cast)

In [6]:
columns = parsed.map(lambda x: x[2]).distinct().collect()
basedict = {metric:0.0 for metric in columns}
ordered = OrderedDict(sorted(basedict.items(), key=lambda y: y[0]))

The two functions below are adapted from user patricksurry's answer to this Stack Overflow question: http://stackoverflow.com/questions/30260015/reshaping-pivoting-data-in-spark-rdd-and-or-spark-dataframes.  Beware, patricksurry's answer is predominantly serial!

In [ ]:
def combine(u1, u2):
  u1.update(u2)
  return u1

def sequential(u, v):
  if not u:
      u = {}
  u[v[2]] = v[3]
  return u

In [ ]:
pivot = parsed.keyBy(lambda row: (row[0], row[1])).aggregateByKey(None, sequential, combine)

In [ ]:
def mergedicts(new, canon):
  tmp = canon.copy()
  tmp.update(new[1])
  return new[0], tmp

In [8]:
final = pivot.map(partial(mergedicts, canon=ordered))

In [56]:
final_ordered = final.takeOrdered(10, key=lambda x: x[0])

In [57]:
final_ordered[0][0]

(1398382545, u'mds01')

In [9]:
final_sorted = final.sortByKey(keyfunc= lambda k: k[0])

In [10]:
final_dict = final_sorted.map(lambda row: row[1].values())

In [11]:
final_dict.coalesce(2).saveAsTextFile("/home/milroy/pyspark/processed.txt")

# Now on to Scala Spark for PCA

In [ ]:
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.linalg.{Vector, Vectors}

val datafilePattern = "/home/milroy/pyspark/processed.txt/part-00000"
val lustreData = sc.textFile(datafilePattern).cache()

val vecData = lustreData.map(line => line.split(",").map(line => line.drop(1).dropRight(1)).map(v => v.toDouble)).map(arr => Vectors.dense(arr))
val rmat: RowMatrix = new RowMatrix(vecData)
val pc: Matrix = rmat.computePrincipalComponents(15)